## Import and preprocess datasets

In [1]:
import numpy as np
from netCDF4 import Dataset
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
from sklearn.ensemble.forest import RandomForestRegressor, RandomForestClassifier
import os
from sklearn import preprocessing, metrics
from importlib import reload
import sknn.mlp
from sklearn import tree  #for graphing random forest tree
import pickle
%matplotlib inline
pylab.rcParams['figure.figsize'] = (10, 6)
inline_rc = dict(mpl.rcParams)
import src.nnload as nnload
import src.nntrain as nntrain
import src.nnplot as nnplot

# Set script parameters
minlev = 0.0
rainonly = False
write_nn_to_netcdf = False
fig_dir = './figs/'
data_dir = './data/'

In [2]:
# Load data
reload(nnload)
x, y, cv, Pout, lat, lev, dlev, timestep = nnload.loaddata(data_dir + 'nntest.nc', minlev,
                                                       rainonly=rainonly) #,all_lats=False,indlat=8)
# Preprocess data
scaler_x, scaler_y, x1, x2, x3, y1, y2, y3, cv1, cv2, cv3 = nnload.pp(x, y, cv, 30000)

/Users/jgdwyer/Dropbox (MIT)/scripts/nn/src/nnload.py:69: UserWarning: Shifting inputs and outputs one time step so they line up!
  warnings.warn("Shifting inputs and outputs one time step so they line up!")


There is some amount of rain 61.0% of the time
There is a rate of >3 mm/day 20.2% of the time
There is convection 65.5% of the time
(819199, 60)
[ 109.14720154]
(819199, 60)


IndexError: too many indices for array

## Create, train, evaluate, and store regressors NN's

In [ ]:
# Build, train, and save model
from importlib import reload
reload(nntrain)
batch_size = 100
r_mlp, r_str                  = nntrain.build_nn('regress',['Rectifier'],[100],'momentum',batch_size,n_stable=25)
                                                # n_stable=25)#, regularize = 'L2', weight_decay = 0.0001)
print(r_str)
r_mlp, r_errors = nntrain.train_nn(r_mlp,r_str,x1,y1,x2,y2)
pickle.dump([r_mlp, r_str, r_errors],  open(data_dir + 'regressors/' + r_str + '_v2.pkl', 'wb'))

# Save model in netcdf format if requested
if write_nn_to_netcdf:
    nnload.write_netcdf_twolayer(r_mlp, 'regress', datadir + 'netcdf_models/' + r_str + '_neural_weights_regressor.nc')

## Compare all NNs to each other

In [ ]:
# Initialize regressors
from importlib import reload
reload(nntrain)
trained_regressors = ['r_600R_mom0.9reg0.0001','r_600R_mom0.9',
                      'r_1200R_mom0.9','r_2400R_mom0.9','r_3600R_mom0.9',
                      'r_4800R_mom0.9']
N_r = len(trained_regressors)
r_mlp = [0]*N_r
r_str = [0]*N_r
r_errors=np.empty((70000,4,N_r))
r_errors[:]=np.NAN

# Load regressors
for i,r in enumerate(trained_regressors):
    r_mlp[i], r_str[i], err = pickle.load(open(data_dir + 'regressors/' + r + '.pkl', 'rb'))
    print(len(err))
    r_errors[:len(err),:,i]      =  err
# Compare them and plot output
nntrain.plot_regressors_scores(r_mlp,r_str,x2,y2,fig_dir + 'compare/','r')
nntrain.plot_model_error_over_time(r_errors, r_str, fig_dir + 'compare/','r')

## Plot test data evaluations

In [ ]:
# Load the model we want to do a detailed evaulation for
reload(nnplot)
r_str_eval = 'r_100R_mom0.9_v2'
r_mlp_eval, _, _ = pickle.load(open(data_dir + 'regressors/' + r_str_eval + '.pkl', 'rb'))

# Set figure path and create directory if it does not exist
figpath = fig_dir + r_str_eval + '/'
if not os.path.exists(figpath):
    os.makedirs(figpath)
    

# Inverse transform output back to physical units
y3_true = scaler_y.inverse_transform(y2)
y3_pred = scaler_y.inverse_transform(r_mlp_eval.predict(x2))

# Plot means and standard deviations
nnplot.plot_means_stds(y3_true, y3_pred, lev, figpath)

# Plot correlation coefficient, explained variance, and rmse
nnplot.plot_error_stats(y3_true, y3_pred, lev, figpath)

# Plot a time series of precipitaiton
nnplot.plot_precip(y3_true, y3_pred, dlev, figpath)

# Plot the enthalpy conservation
nnplot.plot_enthalpy(y3_true, y3_pred, dlev, figpath)

# Plot the rmse vs lat
#nnplot.plot_rmse_vs_lat(r_mlp_eval, figpath, data_dir='./data/',minlev=0.0,rainonly=False)


In [ ]:
reload(nnplot)
reload(nnload)

Tmean, qmean, rmseT, rmseq, rT, rq = nnload.stats_by_latlev(scaler_x,scaler_y,r_mlp_eval,lat, lev)

In [ ]:
reload(nnplot)
nnplot.plot_contour(Tmean,qmean,lat,lev, avg_hem=True)
nnplot.plot_contour(rmseT,rmseq,lat,lev, avg_hem=True)
nnplot.plot_contour(rT, rq, lat, lev, avg_hem=True)

In [ ]:
reload(nnplot)
reload(nnload)
r_str_eval = 'r_100R_mom0.9_v2'
r_mlp_eval, _, _ = pickle.load(open(data_dir + 'regressors/' + r_str_eval + '.pkl', 'rb'))
nnplot.make_contour_plots(figpath, scaler_x,scaler_y,r_mlp_eval,lat, lev)

In [ ]:
b=np.random.rand(3,6)
c=np.reshape(b,-1)
print(c)
d=np.reshape(c,(3,6))
b==d

In [ ]:
a=b.shape
print(a)

In [8]:
def foo(a):
    def bar(b):
        print(b)

In [11]:
foo.bar('hi')

AttributeError: 'function' object has no attribute 'bar'